In [1]:
from parameters import parameters
import numpy as np
import pandas as pd
import Utils
#
# LOOK HERE! 
# This cell must run first to setup the working environment
#

#load the config file
Utils.load_config('config.json')
Utils.print_parameters()

Below are the parameters in use. Check the parameters carefully to ensure a sucussful run.
{ 'agegrid_url': 'https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2016_AREPS/Muller_etal_2016_AREPS_Agegrids/Muller_etal_2016_AREPS_Agegrids_v1.17/Muller_etal_2016_AREPS_v1.17_netCDF/Muller_etal_2016_AREPS_v1.17_AgeGrid-{}.nc',
  'anchor_plate_id': 0,
  'case_name': 'case_AREPS',
  'coastlines': '../data/Global_EarthByte_230-0Ma_GK07_AREPS_Coastlines.gpmlz',
  'convergence_data_dir': './convergence_data/AREPS/',
  'convergence_data_filename_ext': 'csv',
  'convergence_data_filename_prefix': 'subStats',
  'coreg_input_dir': './coreg_input/AREPS/',
  'coreg_input_files': [ 'deposit_candidates.csv',
                         'negative_deposits.csv',
                         'positive_deposits.csv'],
  'coreg_output_dir': 'coreg_output',
  'feature_names': [ 'conv_rate',
                     'dist_nearest_edge',
                     'subduction_volume_km3y',
                     'car

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import pandas as pd
from parameters import parameters 
import Utils as Utils
plt.style.use('seaborn')


names = ["Linear SVM",
         "SVC-POLY",
         #"SVC-POLY-C=100",
         "SVC-RBF",
         #"SVC-RBF-C=100",
         "SVC-SIGM",
         #"SVC-SIGM-C=100",
        #"GaussianProcess-1",
         "GaussianProcess",
         "Decision Tree",
         "RandomForest",
         #"NN-1e-05",
         "Neural_Network",
         "Naive Bayes",
         "QDA",
         "AdaBoost"
         #"AdaBoost-100"
]

classifiers = [SVC(kernel="linear", C=0.025, probability=True),
    SVC(kernel='poly', degree=3, gamma=2, C=10, probability=True),
    #SVC(kernel='poly', degree=3, gamma=2, C=100, probability=True),
    SVC(kernel='rbf', gamma=2, C=10, probability=True),
    #SVC(kernel='rbf', gamma=2, C=100, probability=True),
    SVC(kernel='sigmoid', gamma=2, C=10, probability=True),
    #SVC(kernel='sigmoid', gamma=2, C=100, probability=True),
#    GaussianProcessClassifier(1.0 * RBF(1.0)),
#     GaussianProcessClassifier(2.0 * RBF(1.0)),
    GaussianProcessClassifier(),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    #MLPClassifier(alpha=1e-05, max_iter=1000),
    MLPClassifier(alpha=0.1, max_iter=1000),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
    AdaBoostClassifier(n_estimators=10)]
    #AdaBoostClassifier(n_estimators=100)

# names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
#          "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
#          "Naive Bayes", "QDA"]

# classifiers = [
#     KNeighborsClassifier(3),
#     SVC(kernel="linear", C=0.025, probability=True),
#     SVC(gamma=2, C=1, probability=True),
#     GaussianProcessClassifier(1.0 * RBF(1.0)),
#     DecisionTreeClassifier(max_depth=5),
#     RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
#     MLPClassifier(alpha=1, max_iter=1000),
#     AdaBoostClassifier(),
#     GaussianNB(),
#     QuadraticDiscriminantAnalysis()]



NA_positive = pd.read_csv(Utils.get_ml_input_dir() + 'NA_positive_deposits.csv')
NA_negative = pd.read_csv(Utils.get_ml_input_dir() + 'NA_negative_deposits.csv')

NA_train_test_data = pd.concat([NA_positive, NA_negative])

NA_labels = NA_train_test_data.iloc[:,-1]
NA_data = NA_train_test_data.iloc[:,:-1]

NA_data = preprocessing.scale(NA_data)

In [37]:
train_data, test_data, train_labels, test_labels = train_test_split(NA_data, NA_labels, test_size=0.25, random_state=42)

appended_data = pd.DataFrame()
df = pd.DataFrame()
# appended_data = pd.DataFrame(columns = ['fpr', 'tpr', 'thresh'])
for name, clf in zip(names, classifiers):
    clf.fit(train_data,train_labels)
    pred_prob = clf.predict_proba(test_data)
    fpr, tpr, thresh = roc_curve(test_labels, pred_prob[:,1], pos_label=1)
    random_probs = [0 for i in range(len(test_labels))]
    p_fpr, p_tpr, _ = roc_curve(test_labels, random_probs, pos_label=1)
    auc_score = roc_auc_score(test_labels, pred_prob[:,1])
    df['fpr']=pd.Series(fpr)
    df['tpr']=pd.Series(tpr)
    df['thresh']=pd.Series(thresh)
    df['clf']=name
    df['auc_score'] = auc_score
    appended_data = appended_data.append(df, ignore_index=True)
    
# print(appended_data)
appended_data.to_csv(r'case_AREPS/ml_output/NA_auc-roc.csv')
